In [ ]:
# If running in Colab or a fresh environment
# !pip install transformers datasets scikit-learn torch -q

import numpy as np
import torch

from datasets import Dataset
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer
)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score


In [ ]:
# --------------------------------------------------
# Expanded customer support dataset (50 samples)
# --------------------------------------------------

texts = [
    # ---------- Technical Support (20) ----------
    "My internet connection drops every few minutes",
    "The app crashes when I try to log in",
    "I cannot connect to the server",
    "The website is not loading properly",
    "My screen freezes after the update",
    "The app shows a blank screen",
    "Login fails with an unknown error",
    "The system is very slow today",
    "I keep getting timeout errors",
    "Notifications are not working",
    "The mobile app won’t open",
    "I am unable to reset my password",
    "The software hangs during startup",
    "Audio is not working during calls",
    "The camera feature does not load",
    "The app closes automatically",
    "I get an error when uploading files",
    "The dashboard is not displaying data",
    "My account keeps logging out",
    "The app crashes after the latest patch",

    # ---------- Billing (15) ----------
    "I was charged twice on my credit card",
    "My payment failed but money was deducted",
    "I need a refund for my last payment",
    "The invoice amount is incorrect",
    "I was billed after cancelling my plan",
    "Why was I charged extra this month?",
    "My card was charged without confirmation",
    "I did not receive my refund",
    "Payment status shows pending for days",
    "I was billed even though the service failed",
    "My subscription renewal charge is wrong",
    "I was charged after my free trial ended",
    "Duplicate charge appeared on my statement",
    "The billing date is incorrect",
    "I was overcharged for my subscription",

    # ---------- Sales (15) ----------
    "How can I upgrade my subscription plan?",
    "I want to change to a premium plan",
    "What features are included in the enterprise plan?",
    "Is there a discount for annual billing?",
    "Can I add more users to my account?",
    "What is the price of the pro version?",
    "Do you offer student discounts?",
    "How do I downgrade my plan?",
    "Is there a free trial available?",
    "What plans do you offer?",
    "Can I upgrade in the middle of the month?",
    "What is the difference between basic and premium?",
    "Are there any promotional offers?",
    "How much does the business plan cost?",
    "Can I switch plans anytime?"
]

labels = (
    ["Technical Support"] * 20 +
    ["Billing"] * 15 +
    ["Sales"] * 15
)

print(f"Total samples: {len(texts)}")


Total samples: 50


In [ ]:
# --------------------------------------------------
# Encode labels
# --------------------------------------------------
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# --------------------------------------------------
# Defensive checks (VERY IMPORTANT)
# --------------------------------------------------
assert len(texts) == len(labels_encoded), "Mismatch between texts and labels!"
assert len(set(labels_encoded)) > 1, "Need at least two classes!"

print("Classes:", label_encoder.classes_)


Classes: ['Billing' 'Sales' 'Technical Support']


In [ ]:
# --------------------------------------------------
# Train / Validation split
# --------------------------------------------------
X_train, X_val, y_train, y_val = train_test_split(
    texts,
    labels_encoded,
    test_size=0.2,
    stratify=labels_encoded,
    random_state=42
)

print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")


Training samples: 40
Validation samples: 10


In [ ]:
# --------------------------------------------------
# Load tokenizer
# --------------------------------------------------
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=64
    )

# --------------------------------------------------
# Create Hugging Face datasets
# --------------------------------------------------
train_dataset = Dataset.from_dict({
    "text": X_train,
    "label": y_train
}).map(tokenize, batched=True)

val_dataset = Dataset.from_dict({
    "text": X_val,
    "label": y_val
}).map(tokenize, batched=True)

train_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)

val_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
# --------------------------------------------------
# Load BERT for classification
# --------------------------------------------------
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label_encoder.classes_)
)

print("Model loaded successfully")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully


In [ ]:
# --------------------------------------------------
# Metrics: Accuracy + Macro F1
# --------------------------------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    return {
        "accuracy": accuracy_score(labels, predictions),
        "macro_f1": f1_score(labels, predictions, average="macro")
    }


In [ ]:
# --------------------------------------------------
# Training arguments (compatible with older transformers)
# --------------------------------------------------
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    do_eval=True
)



In [ ]:
# --------------------------------------------------
# Trainer
# --------------------------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-1631270252.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# --------------------------------------------------
# Train
# --------------------------------------------------
trainer.train()


wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.540400


TrainOutput(global_step=15, training_loss=0.509780486424764, metrics={'train_runtime': 146.8439, 'train_samples_per_second': 1.362, 'train_steps_per_second': 0.102, 'total_flos': 6577835443200.0, 'train_loss': 0.509780486424764, 'epoch': 5.0})

In [ ]:
# --------------------------------------------------
# Inference helper
# --------------------------------------------------
def predict_ticket(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=64
    )

    with torch.no_grad():
        outputs = model(**inputs)

    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return label_encoder.inverse_transform([predicted_class])[0]


In [ ]:
# --------------------------------------------------
# Test inference
# --------------------------------------------------
test_text = "My card was billed twice for the same order"
print("Prediction:", predict_ticket(test_text))


Prediction: Technical Support
